In [2]:
!apt-get install git -y

# Use your GitHub account details (not the repo owner's)
!git config --global user.email "bansaltamanna592@gmail.com"
!git config --global user.name "TamannaBansal12"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [9]:
!ls /content


MUFG-hackathon-PHISNET	sample_data


In [4]:
from getpass import getpass
TOKEN = getpass("Enter your GitHub PAT: ")
!git clone https://$TOKEN@github.com/taha-k101/MUFG-hackathon-PHISNET.git
%cd MUFG-hackathon-PHISNET


Enter your GitHub PAT: ··········
Cloning into 'MUFG-hackathon-PHISNET'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 19 (delta 3), reused 12 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (19/19), 1.03 MiB | 6.29 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/MUFG-hackathon-PHISNET


In [ ]:
from google.colab import drive
import zipfile, os

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = "/content/drive/MyDrive"

In [ ]:
print("Files in MyDrive:", os.listdir(DATA_DIR))

Files in MyDrive: ['Colab Notebooks', 'Tech M Project Progress', 'Tamanna_Resumes', 'Imagica Pics', 'Japan_Tech_M', 'MUFG SIH', 'Dataset files_MUFG_SIH.gdoc']


In [ ]:
DATA_DIR = "/content/drive/MyDrive/MUFG SIH"

In [ ]:
print("Files in MUFG SIH:", os.listdir(DATA_DIR))

Files in MUFG SIH: ['email_phishing_dataset.zip', 'Dataset files_MUFG_SIH.gdoc', 'AgenticAI_approach.svg', 'Deepfake_video.zip', 'MUFG', 'SIH2025_Presentation.pptx.pdf', 'Deepfake_audio.zip', 'phishnet']


In [ ]:
text_out  = "/content/email_phishing_dataset"
audio_out = "/content/deepfake_audio_dataset"
video_out = "/content/deepfake_video_dataset"

In [ ]:
os.makedirs(text_out, exist_ok=True)
os.makedirs(audio_out, exist_ok=True)
os.makedirs(video_out, exist_ok=True)

In [ ]:
with zipfile.ZipFile(f"{DATA_DIR}/email_phishing_dataset.zip", 'r') as z:
    z.extractall(text_out)

In [ ]:
with zipfile.ZipFile(f"{DATA_DIR}/Deepfake_audio.zip", 'r') as z:
    z.extractall(audio_out)

In [ ]:
with zipfile.ZipFile(f"{DATA_DIR}/Deepfake_video.zip", 'r') as z:
    z.extractall(video_out)

In [ ]:
print("✅ Extraction complete")
print("Text files:", len(os.listdir(text_out)))
print("Audio files:", len(os.listdir(audio_out)))
print("Video files:", len(os.listdir(video_out)))

✅ Extraction complete
Text files: 7
Audio files: 4
Video files: 2


# **Text Preprocessing**

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")   # 👈 NEW
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re, nltk, unicodedata, os
from nltk.corpus import stopwords

# download all necessary nltk data
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

STOPWORDS = set(stopwords.words("english"))

def preprocess_email(text):
    text = unicodedata.normalize("NFKC", text.lower())
    text = re.sub(r"https?://\S+", "URL", text)   # mask URLs
    text = re.sub(r"[^a-z\s]", " ", text)         # remove non-letters
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if w not in STOPWORDS]
    return " ".join(tokens)

text_dir = "/content/email_phishing_dataset"
processed_texts = {}

for fname in os.listdir(text_dir):
    fpath = os.path.join(text_dir, fname)
    with open(fpath, "r", errors="ignore") as f:
        raw = f.read()
        processed_texts[fname] = preprocess_email(raw)

print("✅ Processed text samples:")
for k,v in list(processed_texts.items())[:2]:
    print(f"{k}: {v[:150]}...")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Processed text samples:
Nigerian_Fraud.csv: sender receiver date subject body urls label mr james ngola james ngola maktoob com webmaster aclweb org thu oct urgent business assistance partnershi...
Enron.csv: subject body label hpl nom may see attached file hplno xls hplno xls nom actual vols th forwarded sabrae zajac hou ect pm enron capital trade resource...


# **Audio Processing**

In [ ]:
import os
print(os.listdir("/content/deepfake_audio_dataset"))


['for-original', 'for-norm', 'for-rerec', 'for-2sec']


In [ ]:
import os, glob, librosa, numpy as np, pandas as pd
from tqdm import tqdm

audio_root = "/content/deepfake_audio_dataset"

# grab common audio types recursively
exts = ["wav","mp3","flac","m4a","ogg","aac","wma"]
audio_files = []
for ext in exts:
    audio_files += glob.glob(f"{audio_root}/**/*.{ext}", recursive=True)

print(f"Found {len(audio_files)} audio files")
if len(audio_files) < 1:
    # quick debug: show tree
    for root, dirs, files in os.walk(audio_root):
        print(root, files[:5])

def subset_from_path(path: str) -> str:
    # returns the immediate subfolder under audio_root, e.g., 'for-original'
    rel = os.path.relpath(path, audio_root)
    parts = rel.split(os.sep)
    return parts[0] if len(parts) > 1 else "root"

def mfcc_mean_vec(path, sr=16000, n_mfcc=13):
    y, sr = librosa.load(path, sr=sr, mono=True)
    # skip super short/empty clips
    if len(y) < sr:   # < 1 second
        return None
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # (n_mfcc, T)
    return mfcc.mean(axis=1)  # (n_mfcc,)

rows = []
skipped = 0
for fpath in tqdm(audio_files):
    try:
        vec = mfcc_mean_vec(fpath)
        if vec is None:
            skipped += 1
            continue
        rows.append({
            "file": os.path.basename(fpath),
            "rel_path": os.path.relpath(fpath, audio_root),
            "subset": subset_from_path(fpath),
            **{f"mfcc_{i+1}": v for i, v in enumerate(vec)}
        })
    except Exception as e:
        skipped += 1
        print(f"⚠️ skip {fpath}: {e}")

df_audio = pd.DataFrame(rows)
save_csv = "/content/drive/MyDrive/processed/processed_audio_mfcc.csv"
os.makedirs(os.path.dirname(save_csv), exist_ok=True)
df_audio.to_csv(save_csv, index=False)

print(f"✅ Saved {len(df_audio)} rows to: {save_csv} | skipped: {skipped}")
print("Counts by subset:\n", df_audio["subset"].value_counts(dropna=False))
df_audio.head()


Found 169754 audio files


 84%|████████▍ | 142532/169754 [38:11<05:22, 84.32it/s]/tmp/ipython-input-2868322402.py:25: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=sr, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
 84%|████████▍ | 142551/169754 [38:11<05:16, 86.00it/s]

⚠️ skip /content/deepfake_audio_dataset/for-original/for-original/training/fake/file32972.mp3: 


 86%|████████▋ | 146770/169754 [39:13<14:25, 26.55it/s]

⚠️ skip /content/deepfake_audio_dataset/for-original/for-original/training/fake/file31017.mp3: 


...

100%|█████████▉| 168942/169754 [44:16<00:09, 89.31it/s]

⚠️ skip /content/deepfake_audio_dataset/for-original/for-original/validation/fake/file31606.mp3: 


100%|██████████| 169754/169754 [44:24<00:00, 63.72it/s]


✅ Saved 163934 rows to: /content/drive/MyDrive/processed/processed_audio_mfcc.csv | skipped: 5820
Counts by subset:
 subset
for-original    68725
for-norm        64071
for-2sec        17870
for-rerec       13268
Name: count, dtype: int64


,file,rel_path,subset,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,file2107.wav,for-original/for-original/testing/real/file210...,for-original,-256.349731,118.939629,10.461822,39.780823,-19.730724,1.610053,0.178933,-19.666792,-3.273839,-5.444699,-14.346134,3.951661,-11.747244
1,file447.wav,for-original/for-original/testing/real/file447...,for-original,-261.060730,56.958260,0.395447,27.690630,-16.021925,2.031633,-11.167014,3.969359,-6.651355,7.207430,-15.071536,2.343977,-3.343426
2,file125.wav,for-original/for-original/testing/real/file125...,for-original,-251.605392,75.884338,11.717957,38.722553,-14.731581,1.523854,-5.391181,2.426846,-8.268317,2.001301,-19.832516,1.563768,-5.711736
3,file1784.wav,for-original/for-original/testing/real/file178...,for-original,-207.510803,95.789398,11.629858,50.834805,-19.259535,4.548469,-9.069361,-21.645176,-7.678545,-6.596760,-10.368109,6.243602,-9.397346
4,file1360.wav,for-original/for-original/testing/real/file136...,for-original,-217.653091,94.334496,1.279012,18.014864,-23.635376,10.071319,-20.874025,-1.791503,-6.930730,4.236859,-13.713974,4.124651,-10.458508


# **Video Processing**

In [ ]:
import os, shutil
from google.colab import drive

# If the folder exists but isn't an active mount, remove it
if os.path.exists('/content/drive') and not os.path.ismount('/content/drive'):
    shutil.rmtree('/content/drive')

os.makedirs('/content/drive', exist_ok=True)
drive.mount('/content/drive', force_remount=True)  # follow the auth link and pick the account with MUFG SIH


Mounted at /content/drive


In [ ]:
import os

# Peek what's actually in your My Drive
print("MyDrive:", os.listdir("/content/drive/MyDrive"))

# Set your MUFG/SIH path (note the SPACE in the folder name)
DATA_DIR = "/content/drive/MyDrive/MUFG SIH"
print("Exists?", os.path.exists(DATA_DIR))
if os.path.exists(DATA_DIR):
    print("MUFG SIH contents:", os.listdir(DATA_DIR))


MyDrive: ['Tech M Project Progress', 'Tamanna_Resumes', 'Imagica Pics', 'Japan_Tech_M', 'MUFG SIH', 'Dataset files_MUFG_SIH.gdoc', 'Colab Notebooks', 'processed']
Exists? True
MUFG SIH contents: ['email_phishing_dataset.zip', 'Dataset files_MUFG_SIH.gdoc', 'AgenticAI_approach.svg', 'Deepfake_video.zip', 'MUFG', 'SIH2025_Presentation.pptx.pdf', 'Deepfake_audio.zip', 'phishnet']


In [ ]:
# ==== VIDEO: extract -> find -> frame-sample -> manifest (simple & stable) ====
!pip install -q opencv-python-headless pandas tqdm

import os, zipfile, shutil, cv2, pandas as pd
from tqdm import tqdm

# ---- paths (confirmed from your Step 1) ----
DATA_DIR   = "/content/drive/MyDrive/MUFG SIH"
VIDEO_ZIP  = os.path.join(DATA_DIR, "Deepfake_video.zip")
VIDEO_ROOT = "/content/deepfake_video_dataset"   # extraction target
FRAMES_ROOT  = "/content/processed_video_frames"
MANIFEST_CSV = "/content/drive/MyDrive/processed/video_frames_manifest.csv"

# ---- clean extract ----
assert os.path.exists(VIDEO_ZIP), f"Zip not found at {VIDEO_ZIP}"
if os.path.exists(VIDEO_ROOT):
    shutil.rmtree(VIDEO_ROOT)
os.makedirs(VIDEO_ROOT, exist_ok=True)
with zipfile.ZipFile(VIDEO_ZIP, "r") as z:
    z.extractall(VIDEO_ROOT)
print("✅ Extracted:", VIDEO_ZIP)
print("Top-level after extract:", os.listdir(VIDEO_ROOT)[:15])

# ---- helpers ----
def list_videos(root):
    exts = (".mp4",".mkv",".mov",".avi",".webm",".m4v",".MP4",".MKV",".MOV",".AVI",".WEBM",".M4V")
    vids=[]
    for dp,_,files in os.walk(root):
        for f in files:
            if f.endswith(exts):
                vids.append(os.path.join(dp,f))
    return vids

# ---- handle nested folder or nested zips if any ----
videos = list_videos(VIDEO_ROOT)
if len(videos) == 0:
    # single nested folder case
    entries = [os.path.join(VIDEO_ROOT, e) for e in os.listdir(VIDEO_ROOT)]
    subdirs = [e for e in entries if os.path.isdir(e)]
    if len(subdirs) == 1:
        VIDEO_ROOT = subdirs[0]
        print("ℹ️ Using nested folder:", VIDEO_ROOT)
        videos = list_videos(VIDEO_ROOT)

if len(videos) == 0:
    # nested zips case
    nested_zips = []
    for dp,_,files in os.walk(VIDEO_ROOT):
        for f in files:
            if f.lower().endswith(".zip"):
                nested_zips.append(os.path.join(dp,f))
    if nested_zips:
        print("ℹ️ Found nested zips; extracting them...")
        for nz in nested_zips:
            try:
                with zipfile.ZipFile(nz, "r") as z:
                    z.extractall(os.path.dirname(nz))
            except Exception as e:
                print("   ⚠️ Could not extract", nz, ":", e)
        videos = list_videos(VIDEO_ROOT)

print(f"🎞 Found {len(videos)} video files under: {VIDEO_ROOT}")
print("Sample:", [v.replace(VIDEO_ROOT + '/', '') for v in videos[:5]])

# ---- stop early if no videos ----
if len(videos) == 0:
    raise SystemExit("❌ No videos found after extraction. Check zip contents or share one filename shown in the folder.")

# ---- frame extraction config ----
FPS_OUT, SIZE, QUALITY = 5, 224, 95
def sample_step(src_fps, fps_out):
    if not src_fps or src_fps <= 0: src_fps = 30.0
    return max(1, int(round(src_fps / fps_out)))
def safe_name(root, path):
    rel = os.path.relpath(path, root)
    base = os.path.splitext(rel)[0]
    return base.replace(os.sep, "__").replace(" ", "_")

os.makedirs(FRAMES_ROOT, exist_ok=True)
os.makedirs(os.path.dirname(MANIFEST_CSV), exist_ok=True)

# ---- process videos -> frames + manifest ----
rows=[]
for vp in tqdm(sorted(videos), desc="extracting frames"):
    cap = cv2.VideoCapture(vp)
    if not cap.isOpened():
        print("⚠️ could not open:", vp);
        continue
    src_fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    step = sample_step(src_fps, FPS_OUT)
    out_dir = os.path.join(FRAMES_ROOT, safe_name(VIDEO_ROOT, vp))
    os.makedirs(out_dir, exist_ok=True)

    idx=saved=0
    while True:
        ok, frame = cap.read()
        if not ok: break
        if idx % step == 0:
            frame = cv2.resize(frame, (SIZE, SIZE))
            cv2.imwrite(os.path.join(out_dir, f"frame_{saved:05d}.jpg"),
                        frame, [int(cv2.IMWRITE_JPEG_QUALITY), QUALITY])
            saved += 1
        idx += 1
    cap.release()

    rows.append({
        "video_rel_path": os.path.relpath(vp, VIDEO_ROOT),
        "frames_dir": out_dir,
        "num_frames": saved,
        "fps_out": FPS_OUT,
        "size": SIZE,
        "src_fps": float(src_fps),
    })

df = pd.DataFrame(rows)
df.to_csv(MANIFEST_CSV, index=False)
print(f"✅ Processed {len(df)} videos | Manifest: {MANIFEST_CSV}")
print(df.head(3))


✅ Extracted: /content/drive/MyDrive/MUFG SIH/Deepfake_video.zip
Top-level after extract: ['DFD_original sequences', 'DFD_manipulated_sequences']
🎞 Found 3431 video files under: /content/deepfake_video_dataset
Sample: ['DFD_original sequences/06__podium_speech_happy.mp4', 'DFD_original sequences/01__kitchen_pan.mp4', 'DFD_original sequences/10__talking_against_wall.mp4', 'DFD_original sequences/11__outside_talking_still_laughing.mp4', 'DFD_original sequences/13__talking_against_wall.mp4']


extracting frames: 100%|██████████| 3431/3431 [7:57:14<00:00,  8.35s/it]


✅ Processed 3431 videos | Manifest: /content/drive/MyDrive/processed/video_frames_manifest.csv
                                      video_rel_path  \
0  DFD_manipulated_sequences/DFD_manipulated_sequ...   
1  DFD_manipulated_sequences/DFD_manipulated_sequ...   
2  DFD_manipulated_sequences/DFD_manipulated_sequ...   

                                          frames_dir  num_frames  fps_out  \
0  /content/processed_video_frames/DFD_manipulate...          42        5   
1  /content/processed_video_frames/DFD_manipulate...         116        5   
2  /content/processed_video_frames/DFD_manipulate...         209        5   

   size  src_fps  
0   224     24.0  
1   224     24.0  
2   224     24.0  
